In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

pip install diffusers transformers accelerate

In [ ]:
print(f"Original video_frames shape: {video_frames.shape}")
print(f"Processed {len(video_frames)} frames, first frame shape: {video_frames[0].shape}")


Original video_frames shape: (1, 200, 256, 256, 3)
Processed 1 frames, first frame shape: (200, 256, 256, 3)


In [ ]:
#pip install --upgrade diffusers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 19.8 MB/s eta 0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.32.1
    Uninstalling diffusers-0.32.1:
      Successfully uninstalled diffusers-0.32.1


In [ ]:
import torch
import diffusers
print(f"Torch version: {torch.__version__}")
print(f"Diffusers version: {diffusers.__version__}")


Torch version: 2.5.1+cu121
Diffusers version: 0.32.2


In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
import numpy as np

# Load pipeline
pipe = DiffusionPipeline.from_pretrained(
    "damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16"
)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Optimize for GPU memory
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()

# Generate video frames
prompt = "A spiderman is surfing"
video_frames = pipe(prompt, num_inference_steps=25, num_frames=200).frames

# Debug output
print(f"Type of video_frames: {type(video_frames)}")
print(f"Shape of video_frames array: {video_frames.shape}")

# Handle batch dimension
if isinstance(video_frames, np.ndarray) and len(video_frames.shape) == 5:
    # Remove batch dimension (shape: (1, 200, 256, 256, 3) -> (200, 256, 256, 3))
    video_frames = video_frames[0]

# Ensure video_frames is a list of frames
if isinstance(video_frames, np.ndarray) and len(video_frames.shape) == 4:
    video_frames = [video_frames[i] for i in range(video_frames.shape[0])]

# Validate and process each frame
processed_frames = []
for i, frame in enumerate(video_frames):
    if len(frame.shape) != 3 or frame.shape[2] not in [1, 3, 4]:
        raise ValueError(f"Frame {i} has an invalid shape: {frame.shape}")
    if frame.shape[2] == 1:  # Convert grayscale to RGB
        frame = np.repeat(frame, 3, axis=2)
    processed_frames.append(frame)

# Export to video
video_path = export_to_video(processed_frames)
print(f"Video saved to {video_path}")


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Type of video_frames: <class 'numpy.ndarray'>
Shape of video_frames array: (1, 200, 256, 256, 3)
Video saved to /tmp/tmpgu7rp6cg.mp4


In [ ]:
video_path

'/tmp/tmpgu7rp6cg.mp4'

In [ ]:
from google.colab import files
files.download(video_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>